<a href="https://colab.research.google.com/github/Realmbird/Recreation-Consitutional-AI/blob/main/Helpful_RLHF_Qwen_0_6B_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==4.0.0 transformers==4.54.1 trl==0.8.2 bitsandbytes==0.46.1 accelerate==1.9.0 peft==0.16.0 huggingface-hub==0.34.1

In [23]:
from datasets import load_dataset, Dataset

In [3]:
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def get_prompts(dataset):
    prompts = []
    for example in dataset:
        chosen_text = example['chosen']
        # Find the end of the human's prompt (before "Assistant:")
        prompt_end_idx = chosen_text.find("\n\nAssistant:")
        if prompt_end_idx != -1:
            prompts.append(chosen_text[:prompt_end_idx].strip())
    return prompts

In [21]:
# prompts had to formated of just queries
test_prompts_list = get_prompts(dataset["train"]) # example just used train not test for promptlist
eval_prompts_list = get_prompts(dataset["test"])
ppo_list = test_prompts_list + eval_prompts_list

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, AutoModelForCausalLM, pipeline
import torch

In [15]:
model_name = "Qwen/Qwen3-0.6B" # Or any other suitable model
mname = model_name
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
#dataset that ppotrainer accepts is tokens
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample)
    return sample


In [24]:
formatted_prompts = [tokenizer(p, return_tensors="pt")['input_ids'][0] for p in ppo_list]
ppo_dataset = Dataset.from_dict({"input_ids": formatted_prompts})

In [25]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

In [26]:
# Important: Add a pad token if the tokenizer doesn't have one, especially for decoder models.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, pad_token_id=tokenizer.pad_token_id, quantization_config=bnb_config) # accepted and rejceted label
# Resize token embeddings if you added a new pad token
model.resize_token_embeddings(len(tokenizer))

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(151669, 1024, padding_idx=151643)

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

In [28]:
from peft import LoraConfig, TaskType, PeftModel, get_peft_model

In [29]:
# stacks on top of BitsAndBytes
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

In [30]:
from trl.trainer.utils import disable_dropout_in_model
from trl import PPOTrainer, PPOConfig
from transformers import TrainingArguments

In [31]:
# Load the reward model directly
reward_model_name = "Realmbird/helpfulness-preference-model-qwen-0.6B-merged"
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name, num_labels=1, device_map="auto") # Added device_map="auto"

In [33]:
ppo_config = PPOConfig(
    learning_rate=1.41e-5,
    batch_size=4,
    mini_batch_size=1,
    gradient_accumulation_steps=4,
)

In [34]:
value_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,
    quantization_config=bnb_config,
    device_map="auto"
)
value_model.resize_token_embeddings(len(tokenizer))

# Apply PEFT to the value model as well
peft_config_value = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
value_model = get_peft_model(value_model, peft_config_value)

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model = ref_model,
    dataset=ppo_dataset,
    tokenizer=tokenizer,
)

ValueError: model must be a PreTrainedModelWrapper, got <class 'transformers.models.qwen3.modeling_qwen3.Qwen3ForCausalLM'> - supported architectures are: (<class 'trl.models.modeling_value_head.AutoModelForCausalLMWithValueHead'>, <class 'trl.models.modeling_value_head.AutoModelForSeq2SeqLMWithValueHead'>)

In [49]:
import trl
print(trl.__version__)

0.20.0
